In [2]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
import pmdarima as pm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import datetime as dt
import datetime

In [7]:
series = pd.read_csv('data.csv', parse_dates=True)
data = series[series['species_name'] == "Rusa alfredi (Visayan Spotted Deer)"]
df=data[['date_report','no_observation']]


#converting 'Date' column to type 'datetime' so that indexing can happen later
df['date_report']=pd.to_datetime(df['date_report'])
df.set_index('date_report')
print (df.dtypes)



date_report       datetime64[ns]
no_observation             int64
dtype: object


In [8]:
df.index

Int64Index([  6,  11,  14,  19,  22,  27,  30,  35,  38,  43,
            ...
            524, 527, 530, 534, 537, 540, 543, 546, 549, 552],
           dtype='int64', length=144)

In [9]:
da = df['date_report'].resample('M').sum()
da.plot(figsize = (15, 6))
plt.show()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'

In [5]:
from pylab import rcParams
rcParams['figure.figsize'] = 18, 8
decomposition = sm.tsa.seasonal_decompose(da, model='additive')
fig = decomposition.plot()
plt.show()

ValueError: You must specify a period or x must be a pandas object with a DatetimeIndex with a freq not set to None

In [ ]:
from statsmodels.tsa.stattools import adfuller
def adf_test(timeseries):
    #Perform Dickey-Fuller test:
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

print(adf_test(da))

In [ ]:
ts_t_adj = da - da.shift(1)
ts_t_adj = ts_t_adj.dropna()
ts_t_adj.plot()

print(adf_test(da))

In [ ]:
ts_s_adj = ts_t_adj - ts_t_adj.shift(4)
ts_s_adj = ts_s_adj.dropna()
ts_s_adj.plot()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
plot_acf(ts_t_adj, lags=46)
matplotlib.pyplot.show()
plot_pacf(ts_t_adj, lags = 22)
matplotlib.pyplot.show()

In [ ]:
p = range(0, 3)
d = range(1,2)
q = range(0, 4)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]
print('Examples of parameter combinations for Seasonal ARIMA...')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
seasonal_pdq

In [ ]:
from pmdarima import auto_arima

Sarimax_model = auto_arima(train_data.Popu,
                       start_p=0,
                       start_q=0,
                       max_p=3,
                       max_q=3,
                       m=12,
                       test='adf',
                       seasonal=True,
                       d=1,
                       D=1,
                       trace=True,
                       error_action='ignore',
                       suppress_warnings=True,
                       stepwise=True)

Sarimax_model.summary()

In [ ]:
#ARIMA(0, 1, 1)x(2, 1, 0, 12)
mod = sm.tsa.statespace.SARIMAX(ts_s_adj,
                                order=(1, 1, 0),
                                seasonal_order=(1, 1, 1, 12))
results = mod.fit(method = 'powell')
print(results.summary().tables[1])

In [ ]:
results.plot_diagnostics(figsize=(18, 8))
plt.show()

In [ ]:
start=pd.to_datetime('2020-12-01')
pred = results.get_prediction(start)
pred_ci = pred.conf_int()
ax = ts_s_adj.plot(label='observed')
pred.predicted_mean.plot(ax=ax, label='One-step ahead Forecast', alpha=.7, figsize=(14, 4))
ax.fill_between(pred_ci.index,
                pred_ci.iloc[:, 0],
                pred_ci.iloc[:, 1], color='k', alpha=.2)
ax.set_xlabel('Date')
ax.set_ylabel('Sightings')
plt.legend()
plt.show()